In [1]:
from keras.applications import ResNet50, VGG16, InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Load pre-trained models
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Custom layers to add on top of the base models
def add_custom_layers(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(10, activation='softmax')(x)  # Adjust for the number of classes
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Adding custom layers
resnet_model = add_custom_layers(resnet_model)
vgg_model = add_custom_layers(vgg_model)
inception_model = add_custom_layers(inception_model)

# Compile models
resnet_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
vgg_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
inception_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 336s 4us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 179s 3us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 314s 4us/step


In [ ]:
# Example training for one model (use the other models in the same way)
resnet_model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=10, batch_size=32)


In [ ]:
import numpy as np

# Make predictions for each model
predictions_resnet = resnet_model.predict(test_data)
predictions_vgg = vgg_model.predict(test_data)
predictions_inception = inception_model.predict(test_data)

# Average the predictions (for regression or probability values)
final_predictions = (predictions_resnet + predictions_vgg + predictions_inception) / 3

# For classification, you can choose the class with the highest probability
final_predictions = np.argmax(final_predictions, axis=1)


In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Step 1: Data Preprocessing
def load_pneumonia_data():
    # Assuming X-ray images are loaded as `images` and labels `labels` (0: normal, 1: pneumonia)
    images = np.array(...)  # Load X-ray images
    labels = np.array(...)  # Load labels

    # Handle missing data
    valid_indices = ~np.isnan(labels)
    images, labels = images[valid_indices], labels[valid_indices]

    # Normalize pixel values to [0, 1]
    images = images.astype('float32') / 255.0

    # Resize to 224x224 for compatibility with InceptionV3
    images = tf.image.resize(images, (224, 224))

    # Split the data into training, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

# Step 2: Data Augmentation
def augment_pneumonia_data(X_train, y_train):
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    train_generator = datagen.flow(X_train, y_train, subset='training', batch_size=32)
    val_generator = datagen.flow(X_train, y_train, subset='validation', batch_size=32)

    return train_generator, val_generator

# Step 3: Model Development using Transfer Learning (InceptionV3)
def build_pneumonia_model():
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze layers for transfer learning

    # Add custom layers for binary classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Step 4: Training the Model
X_train, X_test, y_train, y_test = load_pneumonia_data()
train_generator, val_generator = augment_pneumonia_data(X_train, y_train)

model = build_pneumonia_model()

# Add Callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)
]

history = model.fit(train_generator, epochs=20, validation_data=val_generator, callbacks=callbacks)

# Fine-tuning the Model
base_model.trainable = True
for layer in base_model.layers[:-30]:  # Keep initial layers frozen
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])
history_fine_tune = model.fit(train_generator, validation_data=val_generator, epochs=10)

# Step 5: Evaluation
# Plot training & validation accuracy and loss
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Model Loss')
plt.legend()
plt.show()

# Confusion Matrix
y_pred = (model.predict(X_test) > 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Pneumonia'], yticklabels=['Normal', 'Pneumonia'])
plt.title('Confusion Matrix')
plt.show()

# ROC Curve
y_pred_prob = model.predict(X_test).ravel()
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

# Save the Model
model.save('pneumonia_detection_model.h5')
